In [10]:
import os
import sys
import pandas as pd
import streamlit as st 
import psycopg2
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

from DB_connection.connection import PostgresConnection
#from DB_connection.myconn import read_data
from src.utils import missing_values_table, convert_bytes_to_megabytes




c:\Users\meet_\Documents\GitHub\Telco-database\notebooks
c:\Users\meet_\Documents\GitHub\Telco-database


In [11]:
db = PostgresConnection(dbname='telecom', user='postgres', password='post@5432')
db.connect()

# Example query
query = "SELECT * FROM xdr_data"
result = db.execute_query(query)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df.head())  # Display the first few rows of the DataFrame

# Close the connection when done
db.close_connection()

Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [12]:
missing_values_df = missing_values_table(df)
print("Missing Values in df:")
print(missing_values_df)


Your selected dataframe has 55 columns.
There are 41 columns that have missing values.
Missing Values in df:
                                          Missing Values  % of Total Values  \
Nb of sec with 37500B < Vol UL                    130254               86.8   
Nb of sec with 6250B < Vol UL < 37500B            111843               74.6   
Nb of sec with 125000B < Vol DL                    97538               65.0   
TCP UL Retrans. Vol (Bytes)                        96649               64.4   
Nb of sec with 31250B < Vol DL < 125000B           93586               62.4   
Nb of sec with 1250B < Vol UL < 6250B              92894               61.9   
Nb of sec with 6250B < Vol DL < 31250B             88317               58.9   
TCP DL Retrans. Vol (Bytes)                        88146               58.8   
HTTP UL (Bytes)                                    81810               54.5   
HTTP DL (Bytes)                                    81474               54.3   
Avg RTT DL (ms)       

In [13]:
df['x'] = convert_bytes_to_megabytes(df, 'HTTP DL (Bytes)')

print(df)

           Bearer Id            Start  Start ms              End  End ms  \
0       1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1       1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2       1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3       1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4       1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   
...              ...              ...       ...              ...     ...   
149996  7.277826e+18   4/29/2019 7:28     451.0   4/30/2019 6:02   214.0   
149997  7.349883e+18   4/29/2019 7:28     483.0  4/30/2019 10:41   187.0   
149998  1.311448e+19   4/29/2019 7:28     283.0  4/30/2019 10:46   810.0   
149999  1.311448e+19   4/29/2019 7:28     696.0  4/30/2019 10:40   327.0   
150000           NaN             None       NaN             None     NaN   

        Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0       1823652.0  2.08

In [14]:
#st.title("Tellco data analysis")
#st.write(df)


Data Cleaninig


In [15]:
df.rename(columns={
    'Dur. (ms)': 'session_duration',
    'Total DL (Bytes)': 'total_dl_bytes',
    'Total UL (Bytes)': 'total_ul_bytes'
}, inplace=True)


# Drop rows where essential columns are missing
df.dropna(subset=['MSISDN/Number'], inplace=True)


# Fill missing values
df['session_duration'].fillna(df['session_duration'].mean(), inplace=True)
df['total_dl_bytes'].fillna(0, inplace=True)
df['total_ul_bytes'].fillna(0, inplace=True)


# Create a column for total traffic
df['total_traffic'] = df['total_dl_bytes'] + df['total_ul_bytes']


# Check for duplicates and remove if necessary
df.drop_duplicates(inplace=True)


# Display summary statistics
print(df.describe())


C:\Users\meet_\AppData\Local\Temp\ipykernel_25636\2460049309.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['session_duration'].fillna(df['session_duration'].mean(), inplace=True)
C:\Users\meet_\AppData\Local\Temp\ipykernel_25636\2460049309.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

          Bearer Id       Start ms         End ms  session_duration  \
count  1.485060e+05  148935.000000  148935.000000      1.489350e+05   
mean   1.014676e+19     499.246732     498.675442      1.048702e+05   
std    2.893262e+18     288.555630     288.089136      8.106343e+04   
min    6.917538e+18       0.000000       0.000000      7.142000e+03   
25%    7.349883e+18     250.000000     251.000000      5.788350e+04   
50%    7.349883e+18     499.000000     500.000000      8.639900e+04   
75%    1.304243e+19     749.000000     750.000000      1.327000e+05   
max    1.318654e+19     999.000000     999.000000      1.859336e+06   

               IMSI  MSISDN/Number          IMEI  Avg RTT DL (ms)  \
count  1.489350e+05   1.489350e+05  1.489350e+05    121291.000000   
mean   2.082016e+14   4.188282e+10  4.849493e+13       108.224741   
std    1.522166e+10   2.447443e+12  2.242829e+13       594.299952   
min    2.040471e+14   3.360100e+10  4.400152e+11         0.000000   
25%    2.082014

In [16]:
import matplotlib
matplotlib.use('Qt5Agg')
import seaborn as sns
### 1. Top 10 Handsets Used by Customers
top_handsets = df.groupby('Handset Type')['IMEI'].count().nlargest(10).reset_index()
top_handsets.columns = ['handset_model', 'usage_count']
print("Top 10 Handsets Used by Customers:")
print(top_handsets)


### 2. Top 3 Handset Manufacturers
top_manufacturers = df.groupby('Handset Manufacturer')['IMEI'].count().nlargest(3).reset_index()
top_manufacturers.columns = ['manufacturer', 'usage_count']
print("Top 3 Handset Manufacturers:")
print(top_manufacturers)


### 3. Top 5 Handsets per Top 3 Manufacturers
top_3_manufacturers = top_manufacturers['manufacturer'].tolist()


# Filter the data for the top 3 manufacturers
filtered_df = df[df['Handset Manufacturer'].isin(top_3_manufacturers)]


# Find top 5 handsets for each manufacturer
top_5_handsets_per_manufacturer = (
    filtered_df.groupby(['Handset Manufacturer', 'Handset Type'])['IMEI'].count()
    .groupby(level=0, group_keys=False)
    .nlargest(5).reset_index()
)
top_5_handsets_per_manufacturer.columns = ['manufacturer', 'handset_model', 'usage_count']
print("Top 5 Handsets per Top 3 Manufacturers:")
print(top_5_handsets_per_manufacturer)


### 4. User Behavior on Applications


# Aggregate user behavior per application

"""
app_behavior = df.groupby(['IMEI']).agg(
    total_sessions=('IMEI', 'count'),
    total_session_duration=('Dur. (ms)', 'sum'),
    total_download=('Total DL (Bytes)', 'sum'),
    total_upload=('Total UL (Bytes)', 'sum'),
).reset_index()


print("User Behavior on Applications:")
print(app_behavior.head())
"""



### Visualization of Results
import numpy as np
import matplotlib.pyplot as plt

# a. Top 10 Handsets Used by Customers
plt.figure(figsize=(10, 6))
sns.barplot(x='usage_count', y='handset_model', data=top_handsets)
plt.title('Top 10 Handsets Used by Customers')
plt.xlabel('Usage Count')
plt.ylabel('Handset Model')
plt.show()


# b. Top 3 Handset Manufacturers
plt.figure(figsize=(8, 5))
sns.barplot(x='usage_count', y='manufacturer', data=top_manufacturers)
plt.title('Top 3 Handset Manufacturers')
plt.xlabel('Usage Count')
plt.ylabel('Manufacturer')
plt.show()


# c. Top 5 Handsets per Top 3 Manufacturers
plt.figure(figsize=(12, 8))
sns.barplot(x='usage_count', y='handset_model', hue='manufacturer', data=top_5_handsets_per_manufacturer)
plt.title('Top 5 Handsets per Top 3 Manufacturers')
plt.xlabel('Usage Count')
plt.ylabel('Handset Model')
plt.legend(title='Manufacturer')
plt.show()


Top 10 Handsets Used by Customers:
                  handset_model  usage_count
0              Huawei B528S-23A        19727
1       Apple iPhone 6S (A1688)         9413
2        Apple iPhone 6 (A1586)         9012
3                     undefined         8931
4        Apple iPhone 7 (A1778)         6304
5       Apple iPhone Se (A1723)         5176
6        Apple iPhone 8 (A1905)         4985
7       Apple iPhone Xr (A2105)         4562
8  Samsung Galaxy S8 (Sm-G950F)         4480
9        Apple iPhone X (A1901)         3810
Top 3 Handset Manufacturers:
  manufacturer  usage_count
0        Apple        59464
1      Samsung        40579
2       Huawei        34366
Top 5 Handsets per Top 3 Manufacturers:
   manufacturer                   handset_model  usage_count
0         Apple         Apple iPhone 6S (A1688)         9413
1         Apple          Apple iPhone 6 (A1586)         9012
2         Apple          Apple iPhone 7 (A1778)         6304
3         Apple         Apple iPhone Se (A172